In [1]:
import json
from matplotlib import pyplot as plt
import matplotlib.font_manager as font_manager
import numpy as np

json_file = "../../experiments/exp002_20240907_sna_kadirburak/data-tshark/data_nosnap.json"
with open(json_file) as f:
    data = json.load(f)

# filter data by unique source addresses and get lists

In [2]:
lo_uniqueSAs = []
lo_freqs     = []
lo_SActrs    = []
lo_SSIDs     = []
for pkt in data:
    if(pkt["_source"]["layers"]["wlan.sa"] not in lo_uniqueSAs):
        lo_uniqueSAs.append(pkt["_source"]["layers"]["wlan.sa"])
        lo_freqs.append(pkt["_source"]["layers"]["wlan_radio.frequency"][0])
        lo_SActrs.append(1)
        lo_SSIDs.append(pkt["_source"]["layers"]["wlan.ssid"][0])
    else:
        position_of_uniqueSA_in_listofuniqueSAs = lo_uniqueSAs.index(pkt["_source"]["layers"]["wlan.sa"])
        lo_SActrs[position_of_uniqueSA_in_listofuniqueSAs] += 1

### per-SSID statistics

In [3]:
print("SSID | Source Address    |  Freq |  # of occurrences")
print("-"*50)
KU_AP_idx = []
for i, ssid in enumerate(lo_SSIDs):
    if ssid == "KU":
        print(ssid, "  |", lo_uniqueSAs[i][0], "| ", lo_freqs[i], "| ", lo_SActrs[i])
        KU_AP_idx.append(i)

SSID | Source Address    |  Freq |  # of occurrences
--------------------------------------------------
KU   | 4c:77:6d:92:b3:60 |  2437 |  888
KU   | 4c:77:6d:5f:ea:e0 |  2437 |  890
KU   | 4c:77:6d:a0:45:c0 |  2462 |  899
KU   | 4c:77:6d:5f:e1:e0 |  2462 |  484
KU   | 4c:77:6d:05:af:20 |  2462 |  883
KU   | 4c:77:6d:a0:44:60 |  2412 |  586
KU   | 4c:77:6d:5f:da:c0 |  2412 |  802
KU   | 4c:77:6d:93:1d:21 |  2412 |  275
KU   | 4c:77:6d:9d:e7:e0 |  2412 |  154
KU   | 4c:77:6d:05:a4:00 |  2412 |  865
KU   | 4c:77:6d:96:5b:20 |  2412 |  873
KU   | b4:de:31:f5:7e:00 |  2412 |  278
KU   | 00:fc:ba:7e:0d:e0 |  2437 |  3
KU   | 4c:77:6d:9d:ea:e0 |  2437 |  239
KU   | 20:a5:cb:c8:d5:5d |  2437 |  6
KU   | 0e:8a:ef:72:fc:b5 |  2437 |  11
KU   | 4c:77:6d:96:88:60 |  2412 |  6


In [4]:
KU_AP_idx

[4, 7, 10, 13, 16, 22, 23, 24, 26, 29, 32, 34, 36, 41, 48, 49, 50]

# plotting SSID occurrences

prototype

In [5]:
data[4]

{'_index': 'packets-2024-09-07',
 '_type': 'doc',
 '_score': None,
 '_source': {'layers': {'frame.time': ['18:03:19.879'],
   'wlan_radio.signal_dbm': ['-62'],
   'wlan.sa': ['4c:77:6d:92:b3:60'],
   'wlan.ra': ['ff:ff:ff:ff:ff:ff'],
   'wlan.da': ['ff:ff:ff:ff:ff:ff'],
   'wlan_radio.frequency': ['2437'],
   'wlan.ssid': ['KU'],
   'loc_x': '1.1371889',
   'loc_y': '2.3746476'}}}

In [6]:
def frametime2secs(frametime):
    hrs   = int(frametime.split(":")[0])
    mins  = int(frametime.split(":")[1])
    secs  = int((frametime.split(":")[2]).split(".")[0])
    msecs = int((frametime.split(":")[2]).split(".")[1])
    total_seconds = (hrs*60*60*1000 + mins*60*1000 + secs*1000 + msecs) / 1000.0 
    return total_seconds

def get_timeinsecs(sampledict, offset=0):
    return frametime2secs(sampledict["_source"]["layers"]["frame.time"][0]) - offset;

def get_freq(sampledict):
    return sampledict["_source"]["layers"]["wlan_radio.frequency"][0]

def get_sourceaddr(sampledict):
    return sampledict["_source"]["layers"]["wlan.sa"]
    
def get_ssid(sampledict):
    return sampledict["_source"]["layers"]["wlan.ssid"][0]
    
def get_rss(sampledict):
    return sampledict["_source"]["layers"]["wlan_radio.signal_dbm"][0]

test fcns

In [7]:
print(get_timeinsecs(data[4], offset=64999.800))
print(get_freq(data[4]))
print(get_sourceaddr(data[4])[0])
print(get_ssid(data[4]))
print(get_rss(data[4]))

0.07899999999790452
2437
4c:77:6d:92:b3:60
KU
-62


get the time axis

In [8]:
time = []
for sample in data:
    t = get_timeinsecs(sample, offset=64999.800)
    if(t not in time):
        time.append(t)

### record KU RSS values

first throw out the ones with <300 occurrences: #24, 26, 34, 36, 41, 48, 49, 50

In [9]:
KU_AP_idx_filtered = [x for x in KU_AP_idx if x not in [24, 26, 34, 36, 41, 48, 49, 50]]

In [10]:
lo_uniqueSAs[KU_AP_idx_filtered[-1]]

['4c:77:6d:96:5b:20']

In [11]:
array_ssid_rss = np.zeros((len(time), len(KU_AP_idx_filtered)))*np.nan
for sample in data:
    t_sample  = get_timeinsecs(sample, offset=64999.800)
    t_idx     = time.index(t_sample)
    sa_sample = get_sourceaddr(sample)
    sa_idx    = lo_uniqueSAs.index(sa_sample)
    if(sa_idx in KU_AP_idx_filtered):
        array_ssid_idx = KU_AP_idx_filtered.index(sa_idx)
        array_ssid_rss[t_idx, array_ssid_idx] = get_rss(sample)

In [12]:
array_ssid_rss[:20]

array([[ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [-62.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan, -77.,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [-63.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan, -76.,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,

In [13]:
time[:20]

[0.01599999999598367,
 0.017999999996391125,
 0.021000000000640284,
 0.023999999997613486,
 0.07899999999790452,
 0.08399999999528518,
 0.08899999999994179,
 0.14799999999377178,
 0.1519999999945867,
 0.15799999999580905,
 0.1809999999968568,
 0.18699999999807915,
 0.1919999999954598,
 0.25,
 0.25499999999738066,
 0.2599999999947613,
 0.28299999999580905,
 0.28800000000046566,
 0.2929999999978463,
 0.35199999999895226]

In [ ]:
plt.figure(figsize=(16, 5), dpi=80)
plt.plot(time, array_ssid_rss, marker="*");
plt.ylim([-95, -45])
plt.grid()
plt.legend([(lo_uniqueSAs[idx][0]+ ", count: " + str(lo_SActrs[idx]) + ", freq: " + str(lo_freqs[idx])) for idx in KU_AP_idx_filtered], 
           prop=font_manager.FontProperties(family='monospace'), loc='upper center', ncol=3)

**zoom in a bit**

In [ ]:
fig = plt.figure(figsize=(16, 5), dpi=80)
ax  = fig.add_subplot(1, 1, 1)
cutout = 1000
ax.plot(time[:cutout], array_ssid_rss[:cutout], marker="*");
ax.set_ylim([-95, -50])
ax.set_xticks(np.linspace(0,int(time[cutout]) - 1/2,int(time[cutout])*2), minor=True)
ax.set_xticklabels(np.linspace(0,int(time[cutout]) - 1/2,int(time[cutout])*2), minor=True, rotation=45)
ax.grid(which='both')
ax.legend([(lo_uniqueSAs[idx][0]+ ", count: " + str(lo_SActrs[idx]) + ", freq: " + str(lo_freqs[idx])) for idx in KU_AP_idx_filtered], 
           prop=font_manager.FontProperties(family='monospace'), loc='upper center', ncol=3)